# Проверка на вхождение в список Росфинмониторинга

In [232]:
from datetime import datetime
import pandas as pd
import xml.etree.ElementTree as ET

# Парсинг XML данных 
tree = ET.parse('data/16.12.2024-v 2.1.xml')
root = tree.getroot()

display(root)


<Element 'Перечень' at 0x000002B85720FD30>


АктуальныйПеречень
    Субъект
        ИдСубъекта {}
        ТипСубъекта {}
        История {}
        ФЛ {}
            Идентификатор {}
            Наименование {}
            ДатаВключения {}
            ФИО {}
            Фамилия {}
            Имя {}
            Отчество {}
            ФИОЛат {}
            ДатаРождения {}
            ГодРождения {}
            МестоРождения {}
            СписокДокументов {}
                Документ {}
                    Серия {} 6100
                    Номер {} 326012
                    ОрганВыдачи {} ВЫДАН МО УФМС РФ ПО РОСТОВСКОЙ ОБЛАСТИ В Г. ЗЕРНОГРАДЕ
                    ДатаВыдачи {} 2012-07-21
                    ТипДокумента {} 
                        Идентификатор {} 1631726
                        Наименование {} ПАСПОРТ РФ
                
            СписокГражданств {}
            Адрес {}
            
        СписокАдресов {}
        Примечание {} В РОЗЫСКЕ
        Террорист {} 1 или 0
        

In [ ]:
# Преобразование данных в список словарей 
data = [] 

for child in root:
    if child.tag == 'АктуальныйПеречень':
        for child2 in child:
            if child2.tag == 'Субъект':
                subject_id = None
                remark = None
                terr_marker = None
                fullname = None
                surname = None
                name = None
                secondname = None
                fullname_lat = None
                birth_date = None
                birth_year = None
                birth_place = None
                doc_series = None
                doc_number = None
                doc_issue_department = None
                doc_issue_date = None
                doc_type = None
                append_flag = False
                
                for child3 in child2: # Перебираем субъектов
                    append_flag = False
                    
                    subject_id_subj = child2.find('ИдСубъекта')
                    if not subject_id_subj is None: 
                        subject_id = subject_id_subj.text
                        append_flag = True
                    remark_subj = child2.find('Примечание')
                    if not remark_subj is None: 
                        remark = remark_subj.text
                        append_flag = True
                    terr_marker_subj = child2.find('Террорист')
                    if not terr_marker_subj is None: 
                        terr_marker = terr_marker_subj.text
                        append_flag = True

                    if child3.tag == 'ФЛ':
                        for child4 in child3:
                            if child4.tag == 'ФИО': 
                                fullname = child4.text
                                append_flag = True
                            if child4.tag == 'Фамилия': 
                                surname = child4.text
                                append_flag = True
                            if child4.tag == 'Имя': 
                                name = child4.text
                                append_flag = True
                            if child4.tag == 'Отчество': 
                                secondname = child4.text
                                append_flag = True
                            if child4.tag == 'ФИОЛат': 
                                fullname_lat = child4.text
                                append_flag = True
                            if child4.tag == 'ДатаРождения': 
                                birth_date = child4.text
                                append_flag = True
                            if child4.tag == 'МестоРождения': 
                                birth_place = child4.text
                                append_flag = True
                            if child4.tag == 'ГодРождения': 
                                birth_year = child4.text
                                append_flag = True
                                
                            if child4.tag == 'СписокДокументов':
                        
                                for child5 in child4: # Документ в Списке документов
                                
                                    for child6 in child5:
                                        
                                        if child6.tag == 'Серия': 
                                            doc_series = child6.text
                                            append_flag = True
                                        if child6.tag == 'Номер': 
                                            doc_number = child6.text
                                            append_flag = True
                                        if child6.tag == 'ОрганВыдачи': 
                                            doc_issue_department = child6.text
                                            append_flag = True
                                        if child6.tag == 'ДатаВыдачи': 
                                            doc_issue_date = child6.text
                                            append_flag = True
                                                                                
                                        if child6.tag == 'ТипДокумента':
                                            for child7 in child6:
                                                if child7.tag == 'Наименование': 
                                                    doc_type = child7.text
                                                    append_flag = True
                                
                                    if append_flag :
                                        data.append({
                                            'subject_id': subject_id,
                                            'fullname': fullname, 
                                            'surname': surname,                             
                                            'name': name, 
                                            'secondname': secondname,
                                            'fullname_lat': fullname_lat,
                                            'birth_date': birth_date,
                                            'birth_year': birth_year,
                                            'birth_place': birth_place,
                                            'doc_series': doc_series, 
                                            'doc_number': doc_number, 
                                            'doc_full_number': str(doc_series) + str(doc_number),
                                            'doc_issue_date': doc_issue_date,
                                            'doc_issue_department': doc_issue_department,
                                            'doc_type': doc_type,
                                            'remark': remark,
                                            'terr_marker': terr_marker
                                            })
                                        append_flag = False

                                   
display(data)                      


[{'subject_id': '110354',
  'fullname': 'ДЕРЕВЯНКИН ВАСИЛИЙ ВАСИЛЬЕВИЧ',
  'surname': 'ДЕРЕВЯНКИН',
  'name': 'ВАСИЛИЙ',
  'secondname': 'ВАСИЛЬЕВИЧ',
  'fullname_lat': 'DEREVIANKIN VASILII VASILEVICH',
  'birth_date': '1978-05-27',
  'birth_year': '1978',
  'birth_place': 'Г. ЗЕРНОГРАД РОСТОВСКОЙ ОБЛАСТИ',
  'doc_series': '6002',
  'doc_number': '266659',
  'doc_full_number': '6002266659',
  'doc_issue_date': None,
  'doc_issue_department': None,
  'doc_type': 'ПАСПОРТ РФ',
  'remark': None,
  'terr_marker': '1'},
 {'subject_id': '110354',
  'fullname': 'ДЕРЕВЯНКИН ВАСИЛИЙ ВАСИЛЬЕВИЧ',
  'surname': 'ДЕРЕВЯНКИН',
  'name': 'ВАСИЛИЙ',
  'secondname': 'ВАСИЛЬЕВИЧ',
  'fullname_lat': 'DEREVIANKIN VASILII VASILEVICH',
  'birth_date': '1978-05-27',
  'birth_year': '1978',
  'birth_place': 'Г. ЗЕРНОГРАД РОСТОВСКОЙ ОБЛАСТИ',
  'doc_series': '6100',
  'doc_number': '326012',
  'doc_full_number': '6100326012',
  'doc_issue_date': '2012-07-21',
  'doc_issue_department': 'ВЫДАН МО УФМС РФ ПО РОСТ

In [243]:
# Создание DataFrame из списка словарей 
df = pd.DataFrame(data)

df.head(20)

,subject_id,fullname,surname,name,secondname,fullname_lat,birth_date,birth_year,birth_place,doc_series,doc_number,doc_full_number,doc_issue_date,doc_issue_department,doc_type,remark,terr_marker
0,110354,ДЕРЕВЯНКИН ВАСИЛИЙ ВАСИЛЬЕВИЧ,ДЕРЕВЯНКИН,ВАСИЛИЙ,ВАСИЛЬЕВИЧ,DEREVIANKIN VASILII VASILEVICH,1978-05-27,1978,Г. ЗЕРНОГРАД РОСТОВСКОЙ ОБЛАСТИ,6002,266659,6002266659,None,None,ПАСПОРТ РФ,None,1
1,110354,ДЕРЕВЯНКИН ВАСИЛИЙ ВАСИЛЬЕВИЧ,ДЕРЕВЯНКИН,ВАСИЛИЙ,ВАСИЛЬЕВИЧ,DEREVIANKIN VASILII VASILEVICH,1978-05-27,1978,Г. ЗЕРНОГРАД РОСТОВСКОЙ ОБЛАСТИ,6100,326012,6100326012,2012-07-21,ВЫДАН МО УФМС РФ ПО РОСТОВСКОЙ ОБЛАСТИ В Г. ЗЕ...,ПАСПОРТ РФ,None,1
2,156608,ИГНАТЕНКО ВЯЧЕСЛАВ ВАСИЛЬЕВИЧ,ИГНАТЕНКО,ВЯЧЕСЛАВ,ВАСИЛЬЕВИЧ,IGNATENKO VIACHESLAV VASILEVICH,1967-09-20,1967,Г. РОСТОВ-НА-ДОНУ,6014,660547,6014660547,2014-12-08,ВЫДАН ОУФМС РФ ПО РО В ПРОЛЕТАРСКОМ РАЙОНЕ Г. ...,ПАСПОРТ РФ,None,1
3,156619,ПРЕЛОВСКИЙ ВЛАДИСЛАВ ЕВГЕНЬЕВИЧ,ПРЕЛОВСКИЙ,ВЛАДИСЛАВ,ЕВГЕНЬЕВИЧ,PRELOVSKII VLADISLAV EVGENEVICH,2009-12-27,2009,Г. ИРКУТСК,2523,278244,2523278244,2024-01-16,ВЫДАН ГУ МВД РОССИИ ПО ИРКУТСКОЙ ОБЛАСТИ,ПАСПОРТ РФ,None,1
4,156620,ВТОРУШИН ДМИТРИЙ СЕРГЕЕВИЧ,ВТОРУШИН,ДМИТРИЙ,СЕРГЕЕВИЧ,VTORUSHIN DMITRII SERGEEVICH,1989-05-09,1989,Г. ИРКУТСК,2509,210889,2509210889,2009-05-22,ВЫДАН ОТДЕЛОМ УФМС РОССИИ ПО ИРКУТСКОЙ ОБЛАСТИ...,ПАСПОРТ РФ,None,0
5,156621,КЛИПЦОВ ЮРИЙ ГЕННАДЬЕВИЧ,КЛИПЦОВ,ЮРИЙ,ГЕННАДЬЕВИЧ,KLIPTCOV IURII GENNADEVICH,1993-07-25,1993,С. БАРАБАШ ХАСАНСКОГО РАЙОНА ПРИМОРСКОГО КРАЯ,0513,043396,0513043396,2013-08-14,ВЫДАН ТП №1 ОТДЕЛЕНИЯ УФМС РОССИИ ПО ПРИМОРСКО...,ПАСПОРТ РФ,None,1
6,156622,ДЗИРКВАДЗЕ СТАНИСЛАВ КЕМАЛОВИЧ,ДЗИРКВАДЗЕ,СТАНИСЛАВ,КЕМАЛОВИЧ,DZIRKVADZE STANISLAV KEMALOVICH,1998-05-11,1998,Г. САЯНСК ИРКУТСКОЙ ОБЛАСТИ,2518,575841,2518575841,2018-10-17,ВЫДАН ОТДЕЛОМ ПО ВОПРОСАМ МИГРАЦИИ ОМВД РОССИИ...,ПАСПОРТ РФ,None,0
7,156642,ДЗАРАХОВА НАТАША МАГОМЕДОВНА,ДЗАРАХОВА,НАТАША,МАГОМЕДОВНА,DZARAKHOVA NATASHA MAGOMEDOVNA,1964-03-19,1964,С. МУЖИЧИ РЕСПУБЛИКИ ИНГУШЕТИЯ,0711,581919,0711581919,2011-11-23,ВЫДАН ОТДЕЛОМ УФМС РОССИИ ПО СТАВРОПОЛЬСКОМУ К...,ПАСПОРТ РФ,None,1
8,156643,МУРТУЗОВ ЗАУР БОРИСОВИЧ,МУРТУЗОВ,ЗАУР,БОРИСОВИЧ,MURTUZOV ZAUR BORISOVICH,1977-01-22,1977,Г. БАКУ АЗЕРБАЙДЖАНСКОЙ ССР,4521,685017,4521685017,2022-02-11,ВЫДАН ГУ МВД РОССИИ ПО Г.МОСКВЕ,ПАСПОРТ РФ,None,0
9,156644,ПЕТРОЧЕНКО ДАРЬЯ ВЛАДИМИРОВНА,ПЕТРОЧЕНКО,ДАРЬЯ,ВЛАДИМИРОВНА,PETROCHENKO DARIA VLADIMIROVNA,1992-03-18,1992,С. ТРОИЦКОЕ ЧЕХОВСКОГО РАЙОНА МОСКОВСКОЙ ОБЛАСТИ,4611,743934,4611743934,2012-04-04,ВЫВДАН УПРАВЛЕНИЕМ ФМС РОССИИ ПО МОСКОВСКОЙ ОБ...,ПАСПОРТ РФ,None,0


In [245]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18421 entries, 0 to 18420
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   subject_id            18421 non-null  object
 1   fullname              18421 non-null  object
 2   surname               18420 non-null  object
 3   name                  18414 non-null  object
 4   secondname            18284 non-null  object
 5   fullname_lat          18421 non-null  object
 6   birth_date            18420 non-null  object
 7   birth_year            18421 non-null  object
 8   birth_place           18398 non-null  object
 9   doc_series            17596 non-null  object
 10  doc_number            18420 non-null  object
 11  doc_full_number       18421 non-null  object
 12  doc_issue_date        18048 non-null  object
 13  doc_issue_department  18299 non-null  object
 14  doc_type              18421 non-null  object
 15  remark                6054 non-null 